In [ ]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score,classification_report,f1_score,recall_score,precision_score,roc_curve,auc,roc_auc_score
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
import torch
from torch.utils.data import Dataset,DataLoader
from torch import nn as nn

In [ ]:
df = pd.read_csv('data.csv')
df = df.drop(df.columns[:5], axis = 1)
df.head()

In [ ]:
vc = df['SessionLabel'].value_counts().to_frame().reset_index()
vc['percent'] = vc["SessionLabel"].apply(lambda x : round(100*float(x) / len(df), 2))
vc = vc.rename(columns = {"index" : "Target", "SessionLabel" : "Count"})
vc

In [ ]:
non_stress = df[df['SessionLabel'] == 0].sample(2000)
stress = df[df['SessionLabel'] == 1].sample(300)

data = non_stress.append(stress).sample(frac=1).reset_index(drop=True)
X = data.drop(['SessionLabel'], axis = 1).values
Y = data["SessionLabel"].values

In [ ]:
## input layer
input_layer = Input(shape=(X.shape[1],))

## encoding part
encoded = Dense(100, activation='tanh', activity_regularizer=regularizers.l1(10e-5))(input_layer)
encoded = Dense(50, activation='relu')(encoded)

## decoding part
decoded = Dense(50, activation='tanh')(encoded)
decoded = Dense(100, activation='tanh')(decoded)

## output layer
output_layer = Dense(X.shape[1], activation='relu')(decoded)


In [ ]:
autoencoder = Model(input_layer, output_layer)
autoencoder.compile(optimizer="adadelta", loss="mse")

In [ ]:
x = df.drop(["SessionLabel"], axis=1)
y = df["SessionLabel"].values

x_scale = preprocessing.MinMaxScaler().fit_transform(x.values)
x_nonst, x_stress = x_scale[y == 0], x_scale[y == 1]

In [ ]:
autoencoder.fit(x_nonst[0:200000], x_nonst[0:200000],
                batch_size = 256, epochs = 100,
                shuffle = True, validation_split = 0.20);

In [ ]:
hidden_representation = Sequential()
hidden_representation.add(autoencoder.layers[0])
hidden_representation.add(autoencoder.layers[1])
hidden_representation.add(autoencoder.layers[2])

In [ ]:
nonst_hid_rep = hidden_representation.predict(x_nonst[:500000])
stress_hid_rep = hidden_representation.predict(x_stress[:500000])

In [ ]:
rep_x = np.append(nonst_hid_rep, stress_hid_rep, axis = 0)
y_n = np.zeros(nonst_hid_rep.shape[0])
y_f = np.ones(stress_hid_rep.shape[0])
rep_y = np.append(y_n, y_f)

In [ ]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(max_iter=100)

train_x, val_x, train_y, val_y = train_test_split(rep_x, rep_y, test_size=0.40)


In [ ]:
parameter_space = {
    'hidden_layer_sizes':[(50,50,50), (50,100,50), (100,)],
    'activation':['tanh', 'relu'],
    'solver':['sgd', 'adam'],
    'alpha':[0.0001, 0.05],
    'learning_rate':['constant','adaptive'],
}

In [ ]:
from sklearn.model_selection import GridSearchCV

clf = GridSearchCV(mlp, parameter_space, n_jobs=-1, cv=3)
clf.fit(train_x, train_y)

In [ ]:
# Best paramete set
print('Best parameters found:\n', clf.best_params_)

# All results
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

In [ ]:
y_true, y_pred = val_y , clf.predict(val_x)

from sklearn.metrics import classification_report
print('Results on the test set:')
print(classification_report(y_true, y_pred))
print ("Accuracy Score: ", accuracy_score(y_true, y_pred))

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier

train_x, val_x, train_y, val_y = train_test_split(rep_x, rep_y, test_size=0.40)
#clf = LogisticRegression(solver="lbfgs").fit(train_x, train_y)
clf = MLPClassifier(random_state=1, max_iter=300).fit(train_x, train_y)
#clf = RandomForestClassifier(max_depth=2,random_state=0).fit(train_x,train_y)
#clf= MLPClassifier(solver='adam', activation='relu',alpha=1e-4,hidden_layer_sizes=(50,50,50), random_state=1,max_iter=11,verbose=10,learning_rate_init=.1).fit(train_x, train_y)
pred_y = clf.predict(val_x)

print ("")
print ("Classification Report: ")
print (classification_report(val_y, pred_y))

print ("")
print ("Accuracy Score: ", accuracy_score(val_y, pred_y))

In [ ]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(max_iter=100)



In [ ]:
from sklearn.model_selection import GridSearchCV

clf = GridSearchCV(mlp, parameter_space, n_jobs=-1, cv=3)
clf.fit(DEAP_x_train, DEAP_y_train)

In [ ]:
# Best paramete set
print('Best parameters found:\n', clf.best_params_)

# All results
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

In [ ]:
y_true, y_pred = DEAP_y_test , clf.predict(DEAP_x_test)

from sklearn.metrics import classification_report
print('Results on the test set:')
print(classification_report(y_true, y_pred))